<a href="https://colab.research.google.com/github/ntua-unit-of-control-and-informatics/jaqpot-google-collab-examples/blob/main/Scikit-learn-models/evaluate-a-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluate a Model

In this example, we will demonstrate how to evaluate the robustness of a model using `jaqpotpy`. We will use a RandomForestRegressor model and perform various evaluations including cross-validation, external evaluation, and a randomization test.

In [1]:
# Install `jaqpotpy`
!pip install jaqpotpy

# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from jaqpotpy.models import SklearnModel
from jaqpotpy.datasets import JaqpotTabularDataset
from jaqpotpy.descriptors import RDKitDescriptors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.9/155.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.5/352.5 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/

We start by creating a sample dataset with molecular structures represented as SMILES strings, along with temperature and activity values.

In [2]:
# Create sample data
data = pd.DataFrame(
    {
        "smiles": ["CC", "CCO", "CCC", "CCCl",
        "CCBr", "COC", "CCOCC", "CCCO",
        "CCCC", "CCCCCC",
    ],

        "temperature": np.random.randint(20, 37, size=10),

        "activity": [80, 81, 81, 84, 83.5,
        83, 89, 90, 91, 97,
    ],
    }
)

Next, we prepare the dataset for training using `JaqpotTabularDataset` and `RDKitDescriptors` for feature extraction.

In [4]:
featurizer = RDKitDescriptors()

# Prepare the dataset for training with Jaqpotpy
train_dataset = JaqpotTabularDataset(
    df=data,
    x_cols=["temperature"],
    y_cols=["activity"],
    smiles_cols=["smiles"],
    task="REGRESSION",
    featurizers=featurizer,
)

We then initialize a RandomForestRegressor model and wrap it with `SklearnModel` from `jaqpotpy`. The model is trained on the prepared dataset.

In [5]:
model = RandomForestRegressor(random_state=42)
jaqpot_model = SklearnModel(dataset=train_dataset, model=model)
jaqpot_model.random_seed = 1231
jaqpot_model.fit()

Goodness-of-fit metrics on training set:
{'r2': 0.9476271155760702, 'mae': 0.9069999999999994, 'rmse': 1.2027759558621054}


To estimate the model's performance, we perform cross-validation on the training data.

In [8]:
# Perform cross-validation on the training data
jaqpot_model.cross_validate(train_dataset, n_splits=5)

{'r2': 0.34961769766804024,
 'mae': 2.5109999999999983,
 'rmse': 2.7835837835765496}

We define a test dataset for external evaluation and prepare it using `JaqpotTabularDataset`.

In [9]:
# Define test data for external evaluation
X_test = pd.DataFrame(
    {
        "smiles": ["CCCOC", "CO"],
        "temperature": [27.0, 22.0],
        "activity": [89.0, 86.0],
    }
)

# Prepare the test dataset with Jaqpotpy
test_dataset = JaqpotTabularDataset(
    df=X_test,
    smiles_cols="smiles",
    x_cols=["temperature"],
    y_cols=["activity"],
    task="REGRESSION",
    featurizers=featurizer,
)

We evaluate the model on the test dataset to assess its performance on new/unseen data.

In [10]:
# Evaluate the model on the test dataset
jaqpot_model.evaluate(test_dataset)

predictions = jaqpot_model.predict(test_dataset)
print(predictions)

[89.48 82.01]


Finally, we conduct a randomization test to assess the model's robustness against randomization of target labels.

In [11]:
# Conducts a randomization test to assess the model's robustness
jaqpot_model.randomization_test(
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    n_iters=10,
)

{'iteration_0': {'Train': {'r2': 0.7854517150873384,
   'mae': 1.8369999999999975,
   'rmse': 2.434411633228858},
  'Test': {'r2': -2.113644444444445,
   'mae': 2.6400000000000006,
   'rmse': 2.646828290615015}},
 'iteration_1': {'Train': {'r2': 0.7746349895918184,
   'mae': 1.9780000000000002,
   'rmse': 2.4950240479802988},
  'Test': {'r2': -2.537344444444437,
   'mae': 2.799999999999997,
   'rmse': 2.821174400847984}},
 'iteration_2': {'Train': {'r2': 0.8917207892116936,
   'mae': 1.521499999999999,
   'rmse': 1.7294341560175093},
  'Test': {'r2': -2.381138888888881,
   'mae': 2.052499999999995,
   'rmse': 2.758181012914124}},
 'iteration_3': {'Train': {'r2': 0.8332226445832202,
   'mae': 1.649499999999999,
   'rmse': 2.1463474788579786},
  'Test': {'r2': 0.3735444444444471,
   'mae': 1.1749999999999972,
   'rmse': 1.187234180774793}},
 'iteration_4': {'Train': {'r2': 0.742278124717169,
   'mae': 2.1724999999999994,
   'rmse': 2.668130900087175},
  'Test': {'r2': -10.765694444444456